# Select Repetitions

In [1]:
import functionsMasterProjectMeinhart as fmpm
from matplotlib.widgets import Button, CheckButtons
import pandas as pd
import re
from scipy.signal import butter, lfilter
from os import walk
import shutil

In [2]:
%matplotlib auto

Using matplotlib backend: TkAgg


## Function to filter data

In [3]:

# filter requirements
order = 6 # order of filter
sampling_rate = 256 # [Hz]
cutoff = 10 # cutoff frequency of the filter [Hz]

def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data, axis=0)
    return y


#data_filt_dict = {}
#data_filt_dict['time'] = data_dict['time']           
#for signal in ['Acc','Gyr','Mag']:
#    data_filt_dict[signal] = butter_lowpass_filter(data_dict[signal], cutoff, sampling_rate, order)
    


## Load, Plot, select and save data

In [4]:
# directory of data files
file_dir = 'E:\Physio_Data_Split_Exercise'

# path of file to write selected times
times_file_path = 'E:\Physio_Data_Split_Exercise_done\selected_times.csv'

# destination directory to shift file if selected times are saved
dir_destination = 'E:\Physio_Data_Split_Exercise_done'

subject_nums = ['01','02','03','04','05','06','07','08','09'] # subject 10: no timetable available
exercise_abbrs  = ['RF','RO','RS','LR','BC','TC','MP','SA','P1','P2']
repetitions = ['05','10','15']



fig, axs = plt.subplots(2,1, sharex=True, figsize=(18,9))
plt.subplots_adjust(bottom=0.2, left=0.2) # make space for buttons

# class for marking area via button        
class MarkPlot:

    selected_times = []
    file_name = 'no_file_selected'
    
    selected_exercises = ['RF'] # start exercise
    selected_subjects = ['01'] # start subject
    selected_repetitions = ['05'] # start number of repetition
    
    
    def load_data(self):
        # Load all file names of desired folder
        files = []
        for (dirpath, dirnames, filenames) in walk(file_dir):
            files.extend(filenames)
            break
        
        for ii in range(len(files)):
            # split current file
            current_sub = re.split('[t_.]',files[ii])[1]
            current_ex  = re.split('[t_.]',files[ii])[2]
            current_rep = re.split('[t_.]',files[ii])[3]
            
            # check if file is selected
            if (current_sub in self.selected_subjects) and \
            (current_ex in self.selected_exercises) and (current_rep in self.selected_repetitions):
                self.file_name = files[ii]
                
                file_path = os.path.join(file_dir, self.file_name)
                
                # get data as dictionary
                global data_dict
                data_dict = fmpm.get_sensor_data(in_file=file_path, signals=['Acc','Gyr','Mag'])
                
                # filter data
                global data_filt_dict
                data_filt_dict = {}
                data_filt_dict['time'] = data_dict['time']
                for signal in ['Acc','Gyr','Mag']:
                    data_filt_dict[signal] = butter_lowpass_filter(data_dict[signal], cutoff, sampling_rate, order)
                
                self.new_plot()
                return
        
        self.file_name = 'no_file_selected'
        self.new_plot()

    
    
    def new_plot(self):
        
        if self.file_name == 'no_file_selected':
            fig.suptitle('No file found.', fontsize=18)
            axs[0].clear()
            axs[1].clear()
            plt.draw()
            return
            
        
        fig.suptitle(self.file_name + '\n' + \
                     r'Start selection (button) $\rightarrow$ ' + \
                     r'select points (left click) $\rightarrow$ ' +\
                     r'finish selection (right click) $\rightarrow$ ' +\
                     r'save (button)', fontsize=18)

        plt.sca(axs[0])
        axs[0].clear()
        fmpm.plot_signal(data_filt_dict['Acc'],data_filt_dict['time'],
                         Title='Acceleration Profile')
        plt.gca().set_xlabel('')
        plt.sca(axs[1])
        axs[1].clear()
        fmpm.plot_signal(data_filt_dict['Gyr'],data_filt_dict['time'],
                         Title='Angular Velocity Profile',
                         xLabel=r'$time \enspace [s]$',
                         yLabel=r'$vel \enspace [\frac{deg}{s}]$')
        
        plt.draw()

        
    def unmark(self, event):
        self.selected_times = []
        self.new_plot()

        
    def select_points(self, event):
        
        if self.file_name == 'no_file_selected':
            fig.suptitle(r'Selecting not possibel $\rightarrow$ no file found.', fontsize=18)
            axs[0].clear()
            axs[1].clear()
            plt.draw()
            return
        
        fig.suptitle(self.file_name + '\n' + \
                     r'Start selection (button) $\rightarrow$ ' + \
                     r'select points (left click) $\rightarrow$ ' +\
                     r'finish selection (right click) $\rightarrow$ ' +\
                     r'save (button)', fontsize=18)

        # select points
        selected_points = plt.ginput(-1, mouse_pop=2, mouse_stop=3, show_clicks=True)

        # check number of selected points
        num_points = np.shape(selected_points)[0]
        
        if num_points > 1:
            
            # go through all selected points
            for ii in range(num_points):

                self.selected_times.append(selected_points[ii][0])

                # draw vertical line at selected points
                plt.sca(axs[0])
                plt.axvline(selected_points[ii][0])
                plt.sca(axs[1])
                plt.axvline(selected_points[ii][0])
            
            plt.draw()
            
            
    def save(self, event):
        
        if self.file_name == 'no_file_selected':
            fig.suptitle(r'Saving not possibel $\rightarrow$ no file found.', fontsize=18)
            axs[0].clear()
            axs[1].clear()
            plt.draw()
            return
            
        if not self.selected_times: # if list empty
            fig.suptitle(self.file_name + '\n' + 'No valid selection!', fontsize=18)
        
        else:    
            with open(times_file_path, "a") as time_file:
                time_file.write(self.file_name + '\t' + '\t'.join(str(string) for string in self.selected_times) + '\n')
            
            self.selected_times = []
            
            file_path = os.path.join(file_dir, self.file_name)
            destination_path = os.path.join(dir_destination, self.file_name)
            
            # shift file to other destination folder
            shutil.move(file_path, dir_destination)
            
            # load new data
            self.load_data()
    
    
    def select_exercise(self, label):
        # remove label (exercise abbreviation) from list if it already exists
        if label in self.selected_exercises:
            self.selected_exercises.remove(label)
            
        # otherwise append label (exercise abbreviation)
        else:
            self.selected_exercises.append(label)
        
        self.load_data()
        
        
    def select_subject(self, label):
        # remove label (exercise abbreviation) from list if it already exists
        if label in self.selected_subjects:
            self.selected_subjects.remove(label)
            
        # otherwise append label (exercise abbreviation)
        else:
            self.selected_subjects.append(label)
        
        self.load_data()
        
        
    def select_repetitions(self, label):
        # remove label (exercise abbreviation) from list if it already exists
        if label in self.selected_repetitions:
            self.selected_repetitions.remove(label)
            
        # otherwise append label (exercise abbreviation)
        else:
            self.selected_repetitions.append(label)
            
        self.load_data()
            
    plt.draw()


callback = MarkPlot()
callback.load_data()


# checkbuttons to select exercise
rax_exercise = plt.axes([0.01, 0.48, 0.2, 0.35], frameon=False)
init_state_exercise = np.zeros(len(exercise_abbrs))
init_state_exercise[0] = 1
b_select_exercise = CheckButtons(rax_exercise, exercise_abbrs, init_state_exercise)
b_select_exercise.on_clicked(callback.select_exercise)
fig.text(0.01, 0.87, 'Select Exercises:', fontsize=12)

# checkbuttons to select subject
rax_subject = plt.axes([0.01, 0.014, 0.2, 0.35], frameon=False)
init_state_subject = np.zeros(len(subject_nums))
init_state_subject[0] = 1
b_select_subject = CheckButtons(rax_subject, subject_nums, init_state_subject)
b_select_subject.on_clicked(callback.select_subject)
fig.text(0.01, 0.4, 'Select Subjects:', fontsize=12)

# checkbuttons to select number of repetitions
rax_repetitions = plt.axes([0.1, 0.014, 0.1, 0.13], frameon=False)
init_state_repetitions = np.zeros(len(repetitions))
init_state_repetitions[0] = 1
b_select_repetitions = CheckButtons(rax_repetitions, repetitions, init_state_repetitions)
b_select_repetitions.on_clicked(callback.select_repetitions)
fig.text(0.1, 0.14, 'Select Repetitions:', fontsize=12)

# buttons to unmark, save and select time range
ax_unmark = plt.axes([0.40, 0.05, 0.15, 0.075])
ax_save   = plt.axes([0.57, 0.05, 0.15, 0.075])
ax_select = plt.axes([0.74, 0.05, 0.15, 0.075])

b_unmark = Button(ax_unmark, 'Unmark')
b_unmark.on_clicked(callback.unmark)
b_save = Button(ax_save, 'Save')
b_save.on_clicked(callback.save)
b_select = Button(ax_select, 'Start Selection')
b_select.on_clicked(callback.select_points)

0

In [13]:
files = []
for (dirpath, dirnames, filenames) in walk('E:\Physio_Data_Split'):
    files.extend(filenames)
    break

In [14]:
files

['subject01_BC_05.csv',
 'subject01_BC_10.csv',
 'subject01_BC_15.csv',
 'subject01_LR_05.csv',
 'subject01_LR_10.csv',
 'subject01_LR_15.csv',
 'subject01_MP_05.csv',
 'subject01_MP_10.csv',
 'subject01_MP_15.csv',
 'subject01_P1_05.csv',
 'subject01_P1_10.csv',
 'subject01_P1_15.csv',
 'subject01_P2_05.csv',
 'subject01_P2_10.csv',
 'subject01_P2_15.csv',
 'subject01_RF_05.csv',
 'subject01_RF_10.csv',
 'subject01_RF_15.csv',
 'subject01_RO_05.csv',
 'subject01_RO_10.csv',
 'subject01_RO_15.csv',
 'subject01_RS_05.csv',
 'subject01_RS_10.csv',
 'subject01_RS_15.csv',
 'subject01_SA_05.csv',
 'subject01_SA_10.csv',
 'subject01_SA_15.csv',
 'subject01_TC_05.csv',
 'subject01_TC_10.csv',
 'subject01_TC_15.csv',
 'subject02_BC_05.csv',
 'subject02_BC_10.csv',
 'subject02_BC_15.csv',
 'subject02_LR_05.csv',
 'subject02_LR_10.csv',
 'subject02_LR_15.csv',
 'subject02_MP_05.csv',
 'subject02_MP_10.csv',
 'subject02_MP_15.csv',
 'subject02_P1_05.csv',
 'subject02_P1_10.csv',
 'subject02_P1_1

In [18]:

file_name_test = 'just_a_test.csv'
file_path = os.path.join(file_dir, file_name_test)

dir_destination = 'E:\Physio_Data_Split_done'
destination_path = os.path.join(dir_destination, file_name_test)

# shift file to other folder
shutil.move(file_path, dir_destination)

'E:\\Physio_Data_Split_done\\just_a_test.csv'

In [24]:
list1 = ['1', '2', '3']
'\t'.join(list1)

'1\t2\t3'

In [25]:
L = [1,2,3]

In [53]:
rep_time_data = pd.read_csv(os.path.join('E:\Physio_Data_Split_done','just_a_test.csv'),
                            skiprows=0, sep='|', header=None) # delimiter --> '|' to get one column
rep_time_data.head()

,0
0,subject01_RF_05.csv\t0.11776573700716897\t2.16...
1,subject01_RF_05.csv\t0.05104866711469569\t2.73...
2,subject01_RF_05.csv\t0.03992915546595022\t2.83...
3,subject01_RF_05.csv\t2.91988267249104\t5.67752...
4,subject01_RF_05.csv\t0.42911206317204353\t1.21...


In [57]:
rep_time_data.values[0][0].split('\t')

['subject01_RF_05.csv',
 '0.11776573700716897',
 '2.1637558803763444',
 '5.032589885752689',
 '7.923662914426524',
 '7.923662914426524',
 '7.923662914426524']

In [59]:
float(rep_time_data.values[0][0].split('\t')[1])

0.11776573700716897

In [9]:
file_dir = r'E:\Physio_Data_Split'

files = []
for (dirpath, dirnames, filenames) in walk(file_dir):
    files.extend(filenames)
    break

In [62]:
re.split('[t_.]',files[0])

['subjec', '01', 'BC', '05', 'csv']

In [13]:
files[0].split('_')[1] == 'BC'

True

In [64]:
if 1 is 1 \
and 0 is 0:
    print('Super')

Super


In [84]:
selected_exercises = ['RF'] # start exercise
selected_subjects = ['01'] # start subject
selected_repetitions = ['05'] # start number of repetition

files = []
for (dirpath, dirnames, filenames) in walk(file_dir):
    files.extend(filenames)
    break

for ii in range(len(files)):
    # split current file
    current_sub = re.split('[t_.]',files[ii])[1]
    current_ex  = re.split('[t_.]',files[ii])[2]
    current_rep = re.split('[t_.]',files[ii])[3]

    # check if file is selected
    if (current_sub in selected_subjects) and \
    (current_ex in selected_exercises) and (current_rep in selected_repetitions):
        file_name = files[ii]

        break

file_name

'subject01_RF_05.csv'

In [3]:
list_01 = ['RF']

In [47]:
# remove label (exercise abbreviation) from list if it already exists
label = 'RO'

if label in list_01:
    list_01.remove(label)

# otherwise append label (exercise abbreviation)
else:
    list_01.append(label)
    
list_01

['RS', 'RF', 'RO']